# 1. Importing Libraries

In [1]:
!pip install PyMuPDF
!pip install PyPDF2
!pip install pdfminer.six
!pip install docx2txt
!pip install plotly
!pip install cufflinks
!pip install pyresparser

In [2]:
# importing required modules
import PyPDF2
import pandas as pd
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter  #process_pdf
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
import re
from io import StringIO
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from plotly.offline import init_notebook_mode, iplot
import cufflinks as cf
import spacy
from spacy.matcher import Matcher
import io
from os import listdir
from os.path import isfile, join
import docx2txt
import numpy as np
from wordcloud import WordCloud
from textblob import TextBlob
import plotly.express as px
import os
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

C:\Users\Bhanupriya\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Bhanupriya\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Bhanupriya\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


# 2. Preparing data

In [3]:
text = []
mypath = 'demoresume.docx'
def extract_text_from_doc(doc_path):
    temp = docx2txt.process(doc_path)
    text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
    return ' '.join(text)

resume_data=extract_text_from_doc(mypath)


# 3. Data Cleaning

In [4]:
stemmer = PorterStemmer()
lemmetizer = WordNetLemmatizer()

In [5]:
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     An existing connection was forcibly closed by the
[nltk_data]     remote host>


In [6]:
more_stop_words = ['\x0c','"','-','_','.']
stop_words.extend(more_stop_words)

clean_data() takes sentence separated string input and cleans the data i.e. removing stopwords and punctuation and returning lemmetized string output

In [7]:
def clean_data(text):
    re.sub(r'[\d]','',text)
    re.sub(r'[^a-zA-Z]','',text)
    re.sub('\s+',' ',text)
    text_clean = []
    text_tokens = word_tokenize(text)
     
    for word in text_tokens:
        if (word not in stop_words and 
            word not in string.punctuation): 
            stem_word = lemmetizer.lemmatize(word) 
            text_clean.append(stem_word)
    
    list_to_str = ' '.join([str(ele) for ele in text_clean])
    return list_to_str.lower()

call_to_clean() takes full text as input, breaks them into sentences and calls the clean_data() with the separated sentences as arguments

In [8]:
clean_text = []
def call_to_clean(text):
    sentences = re.split(r'\n+',text)
    sentence_df = pd.DataFrame(sentences, columns = ['Text'])
    sentence_df['Text'] = sentence_df['Text'].apply(clean_data)
    clean_text.append(' '.join(sentence_df['Text']))

In [9]:
import nltk
nltk.download('punkt')
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

call_to_clean(resume_data)

[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>
[nltk_data] Error loading wordnet: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>
[nltk_data] Error loading omw-1.4: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>


In [11]:
clean_text=clean_text[0]

### Cleaned Data

# Feature Extraction

# 1.Name extraction

In [12]:
import spacy
from spacy.matcher import Matcher

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern], None)
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

name =extract_name(clean_text)

 

In [13]:
name

'praveen kumar'

In [15]:
import pandas as pd
parsed_data=pd.DataFrame()

In [57]:
parsed_data["Name"]=pd.Series(name)

# 2.Mobile Number Extraction

In [56]:
import re

def extract_mobile_number(text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
    
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number

mobile =extract_mobile_number(clean_text)
mobile
parsed_data['Mobile']=pd.Series(mobile)

# 3.Email-Id Extraction

In [58]:
def extract_email(email):
    email = re.findall("[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

# Calling the function 
email = extract_email(resume_data)
parsed_data['Email']=pd.Series(email)

In [19]:
skills=pd.read_csv('skills.csv')

In [20]:

skills

,Unnamed: 0,technical skills,ajenti,django-suit,django-xadmin,flask-admin,flower,grappelli,wooey,algorithms,...,virtualized networks,network automation,cloud management,ai,salesforce,mango db,math,calculus,product launch,mvp


# 4.Skills Extraction

In [21]:
from pyresparser import ResumeParser

In [49]:
data_skills = ResumeParser('demoresume.docx').get_extracted_data()['skills']
data_skills= ','.join([str(ele) for ele in data_skills])
data_skills

In [54]:
parsed_data['Skills']=pd.Series(data_skills)

In [62]:
parsed_data

,Name,Mobile,Email,Skills,Education
0,praveen kumar,9704379358,praveen.goshika@musquaretech.com,"Sublime,Engagement,Apis,Ui,Design,Mobile,Postg...",Bachelors Degree (B.Tech) from Avanthi Engine...


# 5.Education Extraction

In [60]:
import spacy
from nltk.corpus import stopwords

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS','Bachelor of Technology','Senior Secondary' 'B.S', 'B.SC', 'B E', 'B. E.','B. E','B S','B. S','B. SC'
            'ME', 'M.E', 'M.E.', 'MS', 'M.S', 'B-TECH','M-TECH','M E', 'M. E','B.COM','B.ED','L.L.B.','LLB','LLM','L.L.M.', 'M. E.', 'M S', 'M. S',
            'BTECH', 'B.TECH', 'M.TECH', 'MTECH','B TECH', 'B. TECH', 'M. TECH', 'M TECH',
            'B. TECH','M. TECH','B TECH','M TECH','M.D.','NDA','N.D.A.','PHD','PGDM','P.G.D.M.' 'MBA','M.B.A.','MCA','M.C.A.','MS','M.S.','MD',
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII',
            'BBA','B.B.A.','BCA','B.C.A.','BA','B.A.', 
        ]


education = []
def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]

    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index]   

    # Extract year
    education = []
    if edu:
        for key in edu.keys():
            year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])  
            if year:
                education.append((key, ''.join(year[0])))
            else:
                education.append(key)
        return education
    else:
        EDU_PATTERN = re.findall(r'Bachelors? \D+|Masters? \D+',resume_text)   
        return EDU_PATTERN
        

# Calling function
education = extract_education(resume_data)

In [61]:
parsed_data['Education']=pd.Series(education)

# 6.Extracting Years of Experience

In [74]:
import numpy as np

In [76]:
def extract_years_of_experience(text):
    lines = sent_tokenize(text)           
    experience = []
    for sentence in lines:
        if re.search('experience',sentence.lower()):        
            sen_tokenized = nltk.word_tokenize(sentence)
            tagged = nltk.pos_tag(sen_tokenized)            
            entities = nltk.chunk.ne_chunk(tagged)          
            for subtree in entities.subtrees():
                for leaf in subtree.leaves():
                    if leaf[1] == 'CD':                     
                        experience.append(leaf[0])
                        
    exp = []
    for ele in experience:
        if len(ele) <=4  or (len(ele) <= 4 and ele[-1] == '0'
                                and ele not in ('2020','2010','2000')):       
            exp.append(ele)
    if exp:
        return exp[0]
    else:
        return np.nan

In [79]:
import spacy
from spacy.matcher import Matcher
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import spacy
from pyresparser import ResumeParser
import numpy as np
import re
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Bhanupriya\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [78]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Bhanupriya\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Bhanupriya\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [80]:
years_of_experience = []

years_of_experience.append(extract_years_of_experience(resume_data))

In [85]:
parsed_data['Experience']=pd.Series(years_of_experience)

# 7.Extraction of Links

In [86]:
links = []
def extract_links(text):
    x = re.findall(r'(https://(www.)?[a-z]+.com(/in)?/[a-z0-9]+)',text)   
    links = []
    for strings in x:
        for link in strings:
            if len(link) > 4:
                links.append(link)
    return links

In [87]:
links=extract_links(resume_data)

In [89]:
parsed_data['links']=pd.Series(links)

In [90]:
parsed_data

,Name,Mobile,Email,Skills,Education,Experience,links
0,praveen kumar,9704379358,praveen.goshika@musquaretech.com,"Sublime,Engagement,Apis,Ui,Design,Mobile,Postg...",Bachelors Degree (B.Tech) from Avanthi Engine...,3yrs,NaN


In [91]:
parsed_json=parsed_data.to_json()
parsed_json

'{"Name":{"0":"praveen kumar"},"Mobile":{"0":"9704379358"},"Email":{"0":"praveen.goshika@musquaretech.com"},"Skills":{"0":"Sublime,Engagement,Apis,Ui,Design,Mobile,Postgresql,Javascript,Outreach,Safety,Testing,Sql,Access,Analysis,Api,Github,Html,Android,Engineering,Controls,Programming,Visual,Health,System,Ai,Css,Healthcare,Database,Website,Js,Technical,Cloud,Windows"},"Education":{"0":"Bachelors Degree (B.Tech) from Avanthi  Engineering College in "},"Experience":{"0":"3yrs"},"links":{"0":null}}'